# LDPC codes basics 
# (with Python example)
## M.Sc. Vladimir Fadeev
### Kazan, 2019

## Introduction

For the first time LDPC codes were described by Robert Gallager in 1963 \[1\], however due to calculation complexity were forgotten for a relatively long time. Only in 1990's these codes were "rediscovered" by M.Davey and D. MacKay who proposed innovative ways how to construct LDPC codes with reduced complexity \[2\]. In fact, LDPC codes are very useful to achieve robustness near to the Shannon limit and also they have lower complexity than Turbo-codes (for higher bit rates such as 3/4, 5/6, 7/8 etc. \[3\]). The central idea of LDPC codes is usage of the pre-initialized parity-check matrices that can be explained via  the Tanner graph representation (fig. 1).

![tanner](https://raw.githubusercontent.com/kirlf/CSP/master/FEC/assets/Tanner_graph.png)
*Fig. 1. Tanner graph representation and its "matrix" form.* 

Parity-check matrix has $(N−K)$ rows and $N$ columns, where $N$ corresponds to desired length of codeword and $K$ corresponds to the length of the message:

$$ \underset{(N-K)\times N} {\mathbf{H}} = \left[ \underset{(N-K)\times K} {\mathbf{P}^T} \qquad  \underset{(N-K)\times (N-K)} {\mathbf{I}}\right] \qquad(1)$$


where $\mathbf{P}$ is the Parity part, and $ \mathbf{I} $ is the identity matrix (standard, systematic form).



## Encoding

Actually, encoding is the simple part of LDPC codes. First of all, a Generator matrix is needed:

$$ \underset{K\times N} {\mathbf{G}} = \left[ \underset{K\times K} {\mathbf{I}} \qquad  \underset{K\times (N-K)} {\mathbf{-P}}\right] \qquad (2)$$

As you may note, the Generator matrix completely depends on the Parity-Check matrix. The encoding procedure is the multiplication of message vector and Generator matrix by modulo $q$, where $q$ is Galois Field parameter: $GF(q = 2^p)$:

$$ \underset{1\times N} {\mathbf{a}} = \underset{1\times K} {\mathbf{u}} \otimes  \underset{K\times N} {\mathbf{G}}\qquad (3)  $$

where $ \mathbf{u} $ is the input message, and $ \mathbf{a} $ is the code word. Obviously, for binary case $q$ is equal to 2.

## Decoding

A lot of decoding algorithms exist for the LDPC codes, but we will consider well-known **sum-product algorithm** (SPA or belief propagation algorithm) [\[4, p.31\]](https://www.researchgate.net/publication/228977165_Introducing_Low-Density_Parity-Check_Codes) with some references to matrix representation during this work. 

First of all, assume that from the channel we have some noisy values that can be represented as soft values after demodulation - Loglikelihood ratios (LLRs):

$$
r = \ln \left( \frac{p(x=0)}{p(x=1)}\right)  = \ln \left(\frac{1 - p}{p} \right) \qquad (4)
$$

where $p$ denotes probability and $x$ denotes some event.

Actually, the decoding procedure can be imagined as transfer of certain probabilities from Variable nodes to Check nodes (V2C message) and vice versa (C2V message). At the first step LLRs correspond to *a priori probabilities*. The SPA aims to maximaze *a posteriori probabilities*, hence the SPA is one of the maximum a posteriori probability (MAP) algorithms.  

### The first iteration (initialization)

The initial point for **V2C** message at the first iteration is the one-rank matrix of the LLRs: 

$$
\underset{M\times N} {\mathbf{M}} = \left(\underset{N\times 1} {\mathbf{r}} \cdot \underset{1\times M} {\mathbf{1}} \right)^T  \odot \underset{M\times N} {\mathbf{H}} \qquad (5)
$$

where $\mathbf{1}$ is the matrix or vector of ones, and $\odot$ denotes Hadamard (element-wise) product. 

> **NOTE #1:**
> 
> One-rank matrix can be replaced by the iterative Hadamard multiplication of the LLR's vector and columns of Parity-Check matrix (additional loop).  

### Variable-to-Check message

Then algorithm requires to process **V2C** message in probability domain using relation between [hyperbolic tangents](http://wwwf.imperial.ac.uk/metric/metric_public/functions_and_graphs/hyperbolic_functions/inverses.html) and natural logarithm [\[4, p.32\]](https://www.researchgate.net/publication/228977165_Introducing_Low-Density_Parity-Check_Codes). Procedure of transmission **V2C** message is multiplication (for probabilities) of non-zero elements in each row:

$$ E_{j,i} = log \left( \frac{1 + \prod_{i'\epsilon B_j, i' \neq i} tanh(M_{j,i'}/2)}{1 - \prod_{i'\epsilon B_j, i' \neq i} tanh(M_{j,i'}/2)} \right) = log \left( \frac{1 + \prod_{i'\epsilon B_j, i' \neq i} M'_{j,i'}}{1 - \prod_{i'\epsilon B_j, i' \neq i} M'_{j,i'}} \right) \qquad(7)$$

where $j$ is the number of the certain row, $i$ is the number of the certain column,   $B_{j}$ is the set of the non-zero elements in $j$-th row, and $i'\neq i$ means that we exlude $i$-th variable node from the consideration.

<img src="https://raw.githubusercontent.com/kirlf/CSP/master/FEC/assets/V2C.png" width="500"/>

*Fig. 2. The illustration of the V2C message passing.*

> **NOTE #2:**
>
> To reduce number of condition checks (such as: zero or non-zero element, i-th element or not) the following mathematical hint can be used:
>
> 1) Replace all of the corresponding to Parity-Check matrix structure zeros by ones: 
> $$ \mathbf{M}'' =  \mathbf{M}' + \mathbf{H}'  $$
> where $\mathbf{H}' = \mathbf{H}+ \mathbf{1}$ is the "mirror" of the Parity-Check matrix
>
> 2) Multiply all of the elements and divide the result by the $i$-th element:
> $$ E_{j,i}' = log \left( \frac {1 + \frac{1}{M"_{j,i}} \prod_{i} M''_{j,i}} {1 - \frac{1}{M"_{j,i}} \prod_{i} M''_{j,i}} \right)  $$
>
> 3) Keep the structure of the Parity-Check matrix:
> $$ \mathbf{E} = \mathbf{E}' \odot \mathbf{H} $$
> 
> This solution can be appropriate for the small study modeling tasks, however, additional calculations may significantly increase simulation time for the large Parity-Check matrices.

### Check-to-Variable message

At the end of the first iteration the LLRs from the channel should be updated. For this purpose we sum up the information of rows in matrix $\mathbf{E}$.

$$
l_i = r_i + \sum_{j\epsilon A_i}E_{j,i} \qquad(8)
$$

where $A_i$ is the set of the coresponding to Parity-Check matrix non-zero elements in $i$-th column. 

> **NOTE #3:**
>
> The summation of all of the column elements can be applied with the same mathematical sense since the zero-elements do not contribute to the addition. 
> $$ \mathbf{l} = \mathbf{r} + \left(\sum\mathbf{E}_{j}\right)^T $$
> The same problems may occure as in the **Note #2**.

After that we map up-to-date LLRs to binary symbol by the rule:

$$
z_i = 
\begin{cases}
0 &\text{if $l_i \geq 0$}\\
1 &\text{if $l_i < 0$}
\end{cases} \qquad(9)
$$

Then necessary condition should be checked:

$$
\underset{M\times 1} {\mathbf{s}} = \underset{M\times N} {\mathbf{H}} \otimes \underset{N\times 1} {\mathbf{z}} = 
\begin{cases}
\mathbf{0} &\text{then stop decoding}\\
\text{non-zero vector} &\text{then continue decoding}
\end{cases} \qquad (10)
$$

If the syndrome $\mathbf{s}$ is not the zero-vector decoding should be continued. Therefore, matrix $\mathbf{M}$ should be recalculated:

$$
M_{j,i} = \sum_{j'\epsilon A_i, j'\neq j} E_{j',i} + r_i \qquad (11)
$$

<img src="https://raw.githubusercontent.com/kirlf/CSP/master/FEC/assets/C2V.png" width="500"/>

*Fig. 3. The illustration of the C2V message passing.*

> **NOTE #4:**
>
> The similar logic as in the **Note 2** and **3** can be applied:
> $$ M_{ij} = r_j - E_{ij} + \sum_{j} E_{ij} $$
> The same problems may occure.



After that the second iteration should follow. Idealy, we have to repeat iterations while $\mathbf{s}$ is a non-zero vector.

# Example

Example is provided according to [\[4, p.33\]](https://www.researchgate.net/publication/228977165_Introducing_Low-Density_Parity-Check_Codes), where \[0., 0., 1., 0., 1., 1.\] message was passed. 

In [1]:
import numpy as np

In [2]:
def sum_prod(r, H):
    stop = False
    Imax = 1000
    I = 0
    H_mirr = (H + np.ones(np.shape(H))) %2
    M = np.zeros(np.shape(H))
    E = np.zeros(np.shape(H))
    l = np.zeros(np.shape(r))
    while stop == False and I != Imax:
        if I == 0:
            for j in range(np.shape(H)[0]):
                M[j, :] = r*H[j, :]
        print('M:\n'+str(M))
        M = np.tanh(M / 2) + H_mirr
        for j in range(np.shape(H)[0]):
            for i in range(np.shape(H)[1]):
                if H[j,i] != 0:
                    E[j,i] = np.log(( 1 + np.prod(M[j,:]) \
                                     / M[j,i]) / ( 1 - np.prod(M[j,:]) / M[j,i]) )
                    
        print('E:\n'+str(E))
        l = r + np.sum(E, axis=0)
        print('l:\n'+str(l))
        for idx, l_j in enumerate(l):
            if l_j >= 0:
                l[idx] = 0
            else:
                l[idx] = 1
        s = np.dot(H, l) %2
        if np.prod(s == np.zeros(np.size(s))) == 1:
            stop = True
        else:
            I = I + 1
            for j in range(np.shape(H)[0]):
                for i in range(np.shape(H)[1]):
                    if H[j,i] != 0:
                        M[j,i] = np.sum(E[:, i]) - E[j,i] + r[j]
    return l
    

In [3]:
#Parity-check matrix (non-systematic form)
H = np.array([[1, 1, 0, 1, 0, 0], [0, 1, 1, 0, 1, 0], [1, 0, 0, 0, 1, 1], [0, 0, 1, 1, 0, 1]]) # non-systematic case

# Received LLRs (the 1st bit is wrong - LDPC code should correct it):
r = np.array([-1.3863, 1.3863, -1.3863, 1.3863, -1.3863, -1.3863])

l = sum_prod(r, H)
print('Decoded message:\n'+str(l))

M:
[[-1.3863  1.3863 -0.      1.3863 -0.     -0.    ]
 [-0.      1.3863 -1.3863  0.     -1.3863 -0.    ]
 [-1.3863  0.     -0.      0.     -1.3863 -1.3863]
 [-0.      0.     -1.3863  1.3863 -0.     -1.3863]]
E:
[[ 0.75377678 -0.75377678  0.         -0.75377678  0.          0.        ]
 [ 0.          0.75377678 -0.75377678  0.         -0.75377678  0.        ]
 [ 0.75377678  0.          0.          0.          0.75377678  0.75377678]
 [ 0.          0.         -0.75377678  0.75377678  0.         -0.75377678]]
l:
[ 0.12125356  1.3863     -2.89385356  1.3863     -1.3863     -1.3863    ]
Decoded message:
[0. 0. 1. 0. 1. 1.]


Well done!

# Non-binary LDPC codes

The non-binary implementation of LDPC codes became popular in recent years (see more via the [following link](https://app.dimensions.ai/analytics/publication/viz/overview-publications?search_text=non-binary%20ldpc&search_type=kws&search_field=full_search)).

Let us count the main challenges of the non-binary case.

The LLRs were represented as vector in case of binary LDPC, however, in fact, LLRs should be represented as a matrix with one zero-vector row (since $ln(1) = 0$). This relates to the certain reference symbol, usually zero, and can be skiped in binary case. However, dimension becomes important in non-binary case (fig. 4).


<img src="https://raw.githubusercontent.com/kirlf/CSP/master/FEC/assets/nonbinLLR.png" width="600" />

*Fig. 4. The matrix of the LLRs*

This means that probabilities of $q-1$ cases should be processed. Each message can be represented as tensor instead of matrix (fig. 5).

<img src="https://raw.githubusercontent.com/kirlf/CSP/master/FEC/assets/C2Vtensor.png" width="500" />

*Fig. 5. tensor representation of the processed message.*

Additionally, weights of the passed should be taken into account (fig. 6).

<img src="https://raw.githubusercontent.com/kirlf/CSP/master/FEC/assets/TannersNonbin.png" width="800" />

*Fig. 6. Tanner graph in non-binary case.*

The indeces of the maximal LLRs can be used as the final decisions (fig. 7).

<img src="https://raw.githubusercontent.com/kirlf/CSP/master/FEC/assets/UptodateLLRnonbin.png" width="600" />

*Fig. 7. The illustration of the decision making.*

All of the considered items require smart solutions to find tradeoffs between processing speed and memory allocation.

# Existing implementations
## (MatLab, Python)

The [**Communication Toolbox**](https://www.mathworks.com/help/comm/block-coding.html) by MathWorks (MatLab) is one of the most famous commertial solutions, however, [open examples](https://www.mathworks.com/matlabcentral/fileexchange/8977-ldpc-code-simulation) are also exist. 

![LDPCMatLab](https://www.mathworks.com/help/examples/comm_product/win64/commDVBS2WithLDPC_03.png)

*Fig. 8. Bit-error ratio performance of 2/3 LDPC code (16APSK, DVB-S.2 Link). Source: ["DVB-S.2 Link, Including LDPC Coding"(MathWorks)](https://www.mathworks.com/help/comm/examples/dvb-s-2-link-including-ldpc-coding.html?s_tid=srchtitle)* 

There are also some open-source implementations in different programming languages including **Python** in the [GitHub](https://github.com/search?o=desc&q=ldpc&s=stars&type=Repositories).

Of course, we should note that these solutions should be additionally reviewed and, maybe, corrected. However, anyway, open-source solutions are great opportunity for the researches. 

# References

1. R.G. Gallager Low-Density Parity-Check Codes, IRE Transactions on Information Theory, 1962
2.  D.J.C. MacKay Good Error-Correcting Codes Based on Very Sparse Matrices, IEEE Transactions on Information Theory, VOL.45, NO 2., March 1999 
3. Andrews, K. S., Divsalar, D., Dolinar, S., Hamkins, J., Jones, C. R., & Pollara, F. (2007). The development of turbo and LDPC codes for deep-space applications. Proceedings of the IEEE, 95(11), 2142-2156.
4. Johnson, S. J. (2006). Introducing low-density parity-check codes. University of Newcastle, Australia, V1.